# Practical 1: Position and Orientation

You may also find these commands useful when using the notebook:
- "Ctrl" + "/" to comment/uncomment
- "Shift" + "Enter" to run the block

In the following code blocks, we will import dependencies and initialize our visualizer

In [1]:
import ipywidgets as widgets
import numpy as np
# from Practical01_Support.NotebookChecker import NotebookChecker
#!pip install meshcat==0.3.2
import meshcat
import meshcat.geometry as g
import meshcat.transformations as tf

#!pip install ece4078
from ece4078.Utility import StartMeshcat
from ece4078.NotebookChecker import NotebookChecker

In [2]:
#!pip uninstall meshcat
#!pip uninstall ece4078

In [3]:
vis = StartMeshcat()


Traceback (most recent call last):
  File "C:\Users\nikhi\anaconda3\Lib\site-packages\numpy\core\__init__.py", line 23, in <module>
    from . import multiarray
  File "C:\Users\nikhi\anaconda3\Lib\site-packages\numpy\core\multiarray.py", line 10, in <module>
    from . import overrides
  File "C:\Users\nikhi\anaconda3\Lib\site-packages\numpy\core\overrides.py", line 8, in <module>
    from numpy.compat._inspect import getargspec
  File "C:\Users\nikhi\anaconda3\Lib\site-packages\numpy\compat\__init__.py", line 12, in <module>
    from . import py3k
  File "C:\Users\nikhi\anaconda3\Lib\site-packages\numpy\compat\py3k.py", line 21, in <module>
    from pathlib import Path
  File "C:\Users\nikhi\anaconda3\Lib\site-packages\pathlib.py", line 10, in <module>
    from collections import Sequence
ImportError: cannot import name 'Sequence' from 'collections' (C:\Users\nikhi\anaconda3\Lib\collections\__init__.py)

During handling of the above exception, another exception occurred:

Traceback 

RuntimeError: the meshcat server process exited prematurely with exit code 1

Testing the Notebook First

To make sure that your notebook instance has been created correctly, please execute the code below.

You should see the plot change as you move the slider.

**FLUX Question**: What is the phrase shown when Omega ($\omega$) =10 at the end?

In [9]:
vis.delete()
scale = 0.75
vis.setPNGView(scale, center = [0.75*scale * lim for lim in [-2, 1, 1, -1]])
notebook_plot, w_slider = NotebookChecker(vis)
display(w_slider, notebook_plot)
display(vis.show_inline())

IntSlider(value=0, continuous_update=False, description='Omega', max=10)

Output()

HTML(value='\n                <div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: hidden; re…

## 1. Rotations in 2D

We define a 2D coordinate frame to represent our robot $(\boldsymbol{x}_1, \boldsymbol{y}_1)$ with respect to the world frame $(\boldsymbol{x}_0, \boldsymbol{y}_0)$. 

Given an angle $\theta$, we describe the relation between the world and robot's frames using the following rotation matrix $R_{01} = \begin{bmatrix}\cos\theta & -\sin\theta \\\sin\theta & \cos\theta \end{bmatrix}$.

Below we show how the robot's frame (green and red arrows) changes relative to the world frame (black) as the angle $\theta$ increases or decreases.

**Interaction**:
- Move the slider to change the rotation angle around the origin

In [ ]:
size = 10

vis.delete()
vis.Set2DView(scale = size, center = [0.75*size * lim for lim in [-2, 1, 1, -1]])

vis["rotated_frame"].set_object(g.triad(size/2))
vis["original_frame"].set_object(g.triad(size/2))
def f(theta):
    a = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
    vis.printMatrix(a, size)
    M = np.identity(4)
    M[:2, :2] = a
    vis["rotated_frame"].set_transform(M)
    return M

slider = widgets.FloatSlider(value=0, min=-2*np.pi, max=2*np.pi, step=0.01, description='Theta',
                                                continuous_update=True)
interactive_plot = widgets.interactive_output(f, {'theta': slider})
display(slider, interactive_plot)
display(vis.show_inline())

## 2. Rotations in 3D

Let us now extend the definition of rotations to a 3-dimensional world.

Recall that

$$R_{x}(\theta) = \begin{bmatrix}1 & 0 & 0 \\ 0 & \cos\theta & -\sin\theta \\ 0 & \sin\theta & \cos\theta \end{bmatrix}, R_{y}(\theta) = \begin{bmatrix} \cos\theta & 0 & \sin\theta \\ 0 & 1 & 0 \\ -\sin\theta & 0 & \cos\theta \end{bmatrix} R_{z}(\theta) = \begin{bmatrix}\cos\theta & -\sin\theta & 0 \\ \sin\theta & \cos\theta & 0 \\ 0 & 0 & 1 \end{bmatrix}$$

**Interaction**:
- Move the sliders to change the rotation angle around each axis
- Click on a button to change the composition order of the rotation matrices

**TO DO**:
- Complete the definition of $R_y(\theta)$
- Implement the $x$-$z$-$y$ and $z$-$x$-$y$ rotation orders

In [ ]:
def rotate3D(mode, theta_x, theta_y, theta_z):  
        
    rot_x = np.identity(3)
    rot_y = np.identity(3)
    rot_z = np.identity(3)
    
    rot_x[1:, 1:] = [[np.cos(theta_x), -np.sin(theta_x)],
                     [np.sin(theta_x), np.cos(theta_x)]]        
    rot_z[0:2, 0:2] = [[np.cos(theta_z), -np.sin(theta_z)],
                      [np.sin(theta_z), np.cos(theta_z)]]
    
    #TODO: Update rot_y ---------------------------------------------
    rot_y[0, :] = [1, 0, 0]
    rot_y[2, :] = [0, 0, 1]
    #ENDTODO ---------------------------------------------
    
    rot_mat = np.identity(3)
    if mode == 'x-z-y':
    #TODO: Complete rotation order x-z-y ----------------------------
        rot_mat = np.eye(3)
    #ENDTODO
    elif mode == 'z-x-y':
    #TODO: Complete rotation order z-x-y-----------------------------
        rot_mat = np.eye(3)
    #ENDTODO
    else:
        rot_mat = rot_x @ rot_z @ rot_y

    M = np.identity(4)
    M[:3, :3] = rot_mat
    print_pos = tf.rotation_matrix(np.pi/2, [0,0,1]) @ tf.translation_matrix([-6, -1, 0])
    vis.printMatrix(rot_mat, 10, print_pos)
    vis["rotated_frame"].set_transform(M)

In [ ]:
vis.delete()
vis.Set3DView(pos = [5.0, 3.0, -3.0])
length = 1
vis.add_thick_triad("rotated_frame", length = length, opacity = 1.0) 
vis.add_thick_triad("original_frame", length = length, opacity = 0.1) 

mode_btn = widgets.ToggleButtons(
  options=['x-z-y', 'z-x-y', 'y-z-x'],
  description='Transform:',
)

stepsize = 0.01

sliders = []
for axis in ["x", "y", "z"]:
    sliders.append(widgets.FloatSlider(value=0, min=-2*np.pi, max=2*np.pi, 
    step=stepsize, description=f'theta_{axis}', continuous_update=True))
widgets.interactive_output(rotate3D, {'mode': mode_btn, 'theta_x': sliders[0], 
                                                    'theta_y': sliders[1], 'theta_z': sliders[2]})
display(mode_btn, *sliders)
display(vis.show_inline())

**FLUX Question**: Are the rotations expressed in fixed or successive frame? 

# 3. Homogeneous Transformations in 3D

Let us now combine rotations and translations in a 3-dimensional world.

Recall that $T_{01} = \begin{vmatrix} R_{01} & \boldsymbol{d}^0_1\\ 0 & 1\end{vmatrix}$, where $R_{01}$ and $\boldsymbol{d}^0_1$ correspond to the rotation and displacement of the robot frame, i.e, *frame 1*, relative to the world frame, i.e., *frame 0*.

**Interaction**:
- Use the button (i.e. x, y, z, transform) and the sliders to visualize the rotation and displacement of the robot frame relative to the corresponding axis in the world frame. Note: if x transform is selected, slider values not associated with x will reset.

**TO DO**:
- Complete the definition of $T_{01}$ along the $z$-axis

In [ ]:
def transform3D (mode, theta_x, d_x, theta_y, d_y, theta_z, d_z):
    trans = np.identity(4)
    rot = np.identity(4)

    if mode == 'x transform':
        trans[0, -1] = d_x
        rot[1:3, 1:3] = [[np.cos(theta_x), -np.sin(theta_x)],
                            [np.sin(theta_x), np.cos(theta_x)]]
    elif mode == 'y transform':
        trans[1,-1] = d_y
        rot[0, 0:3] = [np.cos(theta_y), 0, np.sin(theta_y)]
        rot[2, 0:3] = [-np.sin(theta_y), 0, np.cos(theta_y)]
    elif mode == 'z transform':
        #TODO: Complete definition of z-axis homogeneous transform -----
        pass
        #ENDTODO
    else:
        return -1

    homogeneous_transformation = rot @ trans
    vis["rotated_frame"].set_transform(homogeneous_transformation)
    print_pos = tf.rotation_matrix(np.pi/2, [0,0,1]) @ tf.translation_matrix([-8, 1, 0])
    vis.printMatrix(homogeneous_transformation, 10, print_pos)

In [ ]:
vis.delete()
vis.Set3DView(pos = [5.0, 6.0, -3.0])

length = 1.0
vis.add_thick_triad("rotated_frame", length = length, opacity = 1.0) 
vis.add_thick_triad("original_frame", length = length, opacity = 0.1) 

mode_btn = widgets.ToggleButtons(
  options=['x transform', 'y transform', 'z transform'],
  description='Transform:',
)

stepsize = 0.01

axes = ['x', 'y', 'z']
prefixes = ['theta_', 'd_']
sliders = []
for axis in axes:
    for prefix in prefixes:
        sliders.append(widgets.FloatSlider(
            value = 0, 
            min = -2 if prefix == 'd_' else -2*np.pi, 
            max = 2 if prefix == 'd_' else 2*np.pi,
            step = stepsize,
            description = prefix + axis,
        ))

widgets.interactive_output(transform3D, {'mode': mode_btn, 'theta_x': sliders[0], 'd_x': sliders[1],
                                                'theta_y': sliders[2], 'd_y': sliders[3],
                                                'theta_z': sliders[4], 'd_z': sliders[5]})
display(mode_btn, *sliders)
display(vis.show_inline())